In [4]:
import pandas as pd

/home/marifra/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the key
api_key = os.getenv("OPENAI_API_KEY")

print("API Key loaded:", api_key is not None)

API Key loaded: True


In [22]:
import packaging
from openai import OpenAI
client = OpenAI(api_key = api_key)

### Import and filter dataset based on a minimum number of samples per class

The dataset contains 1588 categories

In [5]:
df_input = pd.read_csv('train.csv')

In [4]:
len(df_input)

20000000

In [5]:
df_input['category'].value_counts()

category
PANTS                                   35973
COFFEE_MAKERS                           35104
BABY_CAR_SEATS                          34163
MUSICAL_KEYBOARDS                       33222
MATTRESSES                              32967
                                        ...  
CONSTRUCTION_LIME_BAGS                    206
COLD_FOOD_AND_DRINK_VENDING_MACHINES      162
PAINTBALL_SMOKE_GRENADES                  154
COMMERCIAL_POPCORN_MACHINES               141
HAMBURGER_FORMERS                         109
Name: count, Length: 1588, dtype: int64

In [6]:
n_sample = 1000
min_samples = 3

# Sample reproducibly
df_sample = df_input.sample(n=n_sample, random_state=42)

# Drop columns
df_sample = df_sample.drop(["language", "label_quality"], axis=1)

# Keep only categories with at least `min_samples`
valid_categories = df_sample['category'].value_counts()
valid_categories = valid_categories[valid_categories >= min_samples].index
df_sample = df_sample[df_sample['category'].isin(valid_categories)]

# Renamdf_input_samplee column
df_sample = df_sample.rename(columns={'title': 'text'})

In [7]:
valid_categories

Index(['CV_JOINTS', 'AIR_COMPRESSORS', 'AUTOMOTIVE_SPRING_SUSPENSIONS',
       'KEYBOARD_CONTROLLERS', 'PANTS', 'CAMERA_AND_CELLPHONE_STABILIZERS',
       'CLEANING_CLOTHS', 'WINES', 'VEHICLE_STICKERS', 'ROLLER_SKATES',
       'CELLPHONE_AND_TABLET_CASES', 'FABRICS', 'SHAVING_MACHINES',
       'GLASSES_FRAMES', 'BOARD_GAMES', 'CD_AND_DVD_PLAYERS', 'SANDPAPERS',
       'PERFUMES', 'ICE_BUCKETS', 'DECORATIVE_PAINTINGS', 'MEN_SWIMWEAR',
       'NETWORK_SWITCHES', 'CANDLES', 'POUFS', 'ELECTRICAL_CABLES',
       'HOME_SHELVES', 'PC_KEYBOARDS', 'VEHICLE_CLUTCH_CABLES',
       'ALTERNATOR_PULLEYS', 'MOTORCYCLE_SEATS',
       'GROUND_AND_WHOLE_BEANS_COFFEE', 'RACQUETS', 'DECORATIVE_VINYLS',
       'EYESHADOWS', 'FISHING_REELS', 'FISHING_RODS', 'WRISTWATCHES',
       'HOME_APPLIANCE_CONTACTORS_AND_RELAYS', 'BRAKE_DRUMS', 'DOLLS',
       'AUDIO_AMPLIFIERS', 'DOORS', 'ALARM_CLOCKS', 'MOTORCYCLE_WHEEL_AXLES',
       'CAR_ENGINE_CAMSHAFTS', 'MALE_UNDERWEAR', 'CLASSIC_CARS',
       'POTENTIOMETERS',

##### Create some examples to use in the prompt

In [8]:
pd.reset_option('display.max_columns') 
pd.reset_option('display.max_rows')

In [7]:
n_examples = 3
top_categories = df_input['category'].value_counts().head(n_examples).reset_index()
examples = df_input[['title', 'category']].drop_duplicates('category', keep='first')
examples_filtered = examples.merge(top_categories[['category']], on= 'category', how='inner')

In [8]:
top_categories

,category,count
0,PANTS,35973
1,COFFEE_MAKERS,35104
2,BABY_CAR_SEATS,34163


In [9]:
few_examples = ""
for row_number, row in examples_filtered.iterrows():
    few_examples += f"Example {row_number+1} '{row['title']}': {row['category']}\n"

In [11]:
print(few_examples)

Example 1 'Butaca  6 Cuotas Sin Interes!! Para Auto Bebesit Hasta 25kg': BABY_CAR_SEATS
Example 2 'Cafetera De Filtro Hamilton Beach 12 Tazas': COFFEE_MAKERS
Example 3 ' Calça Feminina Pra Mulher Gravidas Dois Bolso Cintura Alta ': PANTS



##### Create a function with the prompt to automate the classification request to AI

- System message: high-level, global instructions about the assistant’s role, style, and constraints.

- User message: the actual classification request, including the text to classify and categories.

In [10]:
def create_classification_prompt(text_to_classify, categories):
    category_list_str = ",".join(categories)
    prompt = (
        f"Here are some examples of classifications:\n"
        f"{few_examples}\n"
        f"Now classify the following MercadoLibre product title into one of these categories: {category_list_str}.\n"
        f"Text: \"{text_to_classify}\"\n"
        f"Classification:"
    )
    return prompt

In [12]:
user_inputs = [create_classification_prompt(title, valid_categories) for title in df_sample['text']]

In [16]:
len(valid_categories), len(df_sample), len(user_inputs[1])

(73, 247, 1583)

In [17]:
user_inputs[:10]

['Here are some examples of classifications:\nExample 1 \'Butaca  6 Cuotas Sin Interes!! Para Auto Bebesit Hasta 25kg\': BABY_CAR_SEATS\nExample 2 \'Cafetera De Filtro Hamilton Beach 12 Tazas\': COFFEE_MAKERS\nExample 3 \' Calça Feminina Pra Mulher Gravidas Dois Bolso Cintura Alta \': PANTS\n\nNow classify the following MercadoLibre product title into one of these categories: CV_JOINTS,AIR_COMPRESSORS,AUTOMOTIVE_SPRING_SUSPENSIONS,KEYBOARD_CONTROLLERS,PANTS,CAMERA_AND_CELLPHONE_STABILIZERS,CLEANING_CLOTHS,WINES,VEHICLE_STICKERS,ROLLER_SKATES,CELLPHONE_AND_TABLET_CASES,FABRICS,SHAVING_MACHINES,GLASSES_FRAMES,BOARD_GAMES,CD_AND_DVD_PLAYERS,SANDPAPERS,PERFUMES,ICE_BUCKETS,DECORATIVE_PAINTINGS,MEN_SWIMWEAR,NETWORK_SWITCHES,CANDLES,POUFS,ELECTRICAL_CABLES,HOME_SHELVES,PC_KEYBOARDS,VEHICLE_CLUTCH_CABLES,ALTERNATOR_PULLEYS,MOTORCYCLE_SEATS,GROUND_AND_WHOLE_BEANS_COFFEE,RACQUETS,DECORATIVE_VINYLS,EYESHADOWS,FISHING_REELS,FISHING_RODS,WRISTWATCHES,HOME_APPLIANCE_CONTACTORS_AND_RELAYS,BRAKE_DRUM

#### How many tokens does one of the input have?

In [13]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

text = user_inputs[1]
categories_str = ", ".join(valid_categories)
tokens_text = encoding.encode(text)
tokens_categories = encoding.encode(categories_str)

print("Number of Categories:", len(valid_categories))
print("Tokens of Categories", len(tokens_categories))
print("Tokens of Full Text (instructions + examples + categories + product title):", len(tokens_text))

Number of Categories: 73
Tokens of Categories 454
Tokens of Full Text (instructions + examples + categories + product title): 569


##### Cost Estimation

247 records with 73 categories → 569 tokens per record

569×247=140,543 tokens

| Model       | Price per 1k tokens | Cost                                    |
| ----------- | ------------------- | --------------------------------------- |
| GPT-5-nano  | $0.0001             | 140,543 ÷ 1,000 × 0.0001 ≈ **$0.014**   |
| GPT-4o-mini | $0.15 / 1M          | 140,543 ÷ 1,000,000 × 0.15 ≈ **$0.021** |
| GPT-4o      | $2.50 / 1M          | 140,543 ÷ 1,000,000 × 2.50 ≈ **$0.35**  |
| GPT-4-turbo | $10 / 1M            | 140,543 ÷ 1,000,000 × 10 ≈ **$1.41**    |


The whole dataset:

1588 categories: 1,588 × 6.22 ≈ 9,874 tokens for categories

Text/instructions part: 9,874 + 115 ≈ 9,989 ≈10,000 tokens per record


Total tokens for 20,000,000 records ≈ 10,000×20,000,000 = 200,000,000,000 tokens

**Cost for 20M records** ≈ 200,000,000,000 ÷ 1,000 × 0.0001 = **$20,000** 😱 


##### Create a function to build the message in the expected format by the OpenAI Chat API and another to call the model and extract the responses

The message has two roles: system and user

In [14]:
def format_message(role, content):
    return {"role": role, "content": content}


def get_response(messages):
    completion = client.chat.completions.create(
        model="gpt-5-nano", #-- 'gpt-3.5-turbo',
        messages=messages,
    )
    content = completion.choices[0].message.content
    return content

In [15]:
system_message = (
"You are a product classifier for MercadoLibre product titles. "
"Titles may be in Spanish or Portuguese. "
"Always classify them into one of the provided categories, which are in English. "
"Return only the category name, without explanations or extra text. "
"If no category seems perfect, choose the closest match from the list. "
"Do not invent new categories."
)

##### Set messages using the system_message and the user_inputs as output of create_classification_prompt() function.

In [23]:
responses = []
for user_input in user_inputs:
    messages = [format_message("system", system_message),
                format_message("user", user_input)]
    response = get_response(messages)
    responses.append(response)

In [24]:
responses

['CLASSIC_CARS',
 'MOTORCYCLE_IGNITION_SWITCHES',
 'NETWORK_SWITCHES',
 'CELLPHONES',
 'AUTOMOTIVE_BATTERIES',
 'AUTOMOTIVE_WATER_PUMPS',
 'FABRICS',
 'NETWORK_SWITCHES',
 'SEAT_BELTS',
 'CLEANING_CLOTHS',
 'PUZZLE_CUBES',
 'CV_JOINTS',
 'AUDIO_AMPLIFIERS',
 'ELECTRICAL_CABLES',
 'CAMERA_AND_CELLPHONE_STABILIZERS',
 'MOTORCYCLE_IGNITION_SWITCHES',
 'SANDPAPERS',
 'EYESHADOWS',
 'DECORATIVE_VINYLS',
 'AIR_COMPRESSORS',
 'WINES',
 'WINES',
 'CAMERA_AND_CELLPHONE_STABILIZERS',
 'MOTORCYCLE_SEATS',
 'CLASSIC_CARS',
 'VEHICLE_STICKERS',
 'VEHICLE_CLUTCH_CABLES',
 'ROLLER_SKATES',
 'PERFUMES',
 'DOLLS',
 'RACQUETS',
 'DOORS',
 'SHAVING_MACHINES',
 'GLASSES_FRAMES',
 'PORTABLE_CELLPHONE_CHARGERS',
 'KEYBOARD_CONTROLLERS',
 'WHEELS_BEARINGS',
 'KEYBOARD_CONTROLLERS',
 'CV_JOINTS',
 'VIBRATORS',
 'PROTECTION_NETS',
 'PC_KEYBOARDS',
 'CD_AND_DVD_PLAYERS',
 'CV_JOINTS',
 'KITCHEN_BOWLS',
 'CAMPING_STOVES',
 'ALARM_CLOCKS',
 'GARDEN_HOSES',
 'ENGINE_OIL_PRESSURE_SENSORS',
 'AUDIO_AMPLIFIERS',
 'PE

In [27]:
df_sample['prediction'] = responses

In [39]:
df_sample.head()

,text,category,predictions,prediction
19555084,Nissan Versa,CLASSIC_CARS,CLASSIC_CARS,CLASSIC_CARS
831796,Llave Contacto Yamaha Fz 16 + Tapa Tanque Mpr,MOTORCYCLE_IGNITION_SWITCHES,MOTORCYCLE_IGNITION_SWITCHES,MOTORCYCLE_IGNITION_SWITCHES
6623530,Switch Ubiquiti Unifi 8 - Puertos Giga Admini...,NETWORK_SWITCHES,NETWORK_SWITCHES,NETWORK_SWITCHES
8310326,Moto G5 Libre Huella Zona Oeste,CELLPHONES,CELLPHONES,CELLPHONES
14784082,Bateria Heliar 50 Jd *** New Civic- Crv***bate...,AUTOMOTIVE_BATTERIES,AUTOMOTIVE_BATTERIES,AUTOMOTIVE_BATTERIES


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print("Accuracy: ", accuracy_score(df_sample['category'], df_sample['prediction']))
print("F1 Score: ", f1_score(df_sample['category'], df_sample['prediction'], average='macro')) #macro: (treat all classes equally)

Accuracy:  0.8663967611336032
F1 Score:  0.8443247443247444


? Fine Tuning GPT: This process adapts a pre-trained GPT model to a specific task by training it on your labeled data.